In [1]:
import pandas as pd
from pprint import pprint
import os

# Paths according to the OS
if os.name == 'posix':
    fixture_coaff_path = '/home/kevin/simplon/briefs/avv-matcher/processing_data/datas/fixtures_coaff.csv'
    fixture_psarm_path = '/home/kevin/simplon/briefs/avv-matcher/processing_data/datas/fixtures_psarm.csv'
    combined_result_path = '/home/kevin/simplon/briefs/avv-matcher/processing_data/datas/combined_result.csv'
else:
    fixture_coaff_path = r'C:\Users\thibaut.boguszewski\Desktop\avv-matcher\processing_data\datas\fixtures_coaff.csv'
    fixture_psarm_path = r'C:\Users\thibaut.boguszewski\Desktop\avv-matcher\processing_data\datas\fixtures_psarm.csv'
    combined_result_path = r'C:\Users\thibaut.boguszewski\Desktop\avv-matcher\processing_data\datas\combined_result.csv'

coaff_df = pd.read_csv(fixture_coaff_path)

coaff_df.head(10)

,Localisation,COEFF_F212,PROFIL,Nom,Missions_en_cours,Competences,Date_Demarrage,Date_de_fin,Tx_occup,Stream_BT
0,Tours,150,Sénior,Sharon Cooper,Bouygues,Data Ing + Data S.,2023-07-25,2023-09-13,0.8,TALEND
1,Tours,150,Sénior,Sharon Cooper,Engie,Data Ing + Data S.,2023-06-25,2024-03-02,0.2,TALEND
2,Tours,150,Sénior,Sharon Cooper,Club Med,Data Ing + Data S.,2023-12-18,2024-04-15,0.8,TALEND
3,Tours,100,Expert,Matthew Wilson,DISPO ICE,Azure,2024-05-17,2025-01-25,0.2,Cloud Azure
4,Tours,100,Expert,Matthew Wilson,Club Med,Azure,2024-12-28,2025-09-17,0.8,Cloud Azure
5,Tours,100,Expert,Matthew Wilson,La Poste,Azure,2025-08-08,2025-10-21,0.2,Cloud Azure
6,Tours,100,Expert,Matthew Wilson,Dalkia,Azure,2025-08-24,2026-01-21,0.8,Cloud Azure
7,Tours,100,Expert,Matthew Wilson,Bouygues,Azure,2025-11-10,2026-07-26,0.2,Cloud Azure
8,Paris,105,Junior,Robert Trujillo,Club Med,Data Ing + Data S.,2024-06-26,2025-06-16,0.8,DataViz
9,Paris,105,Junior,Robert Trujillo,Orange,Data Ing + Data S.,2025-04-09,2025-05-12,0.2,DataViz


In [2]:
psarm_df = pd.read_csv(fixture_psarm_path)

psarm_df.head(10)

,Code,Nom,Stat. Pers.,Type,HR Business Unit,Dept Descr,Executive BU,CGI Groupe d'affaire,CGI BU,Statut employé,...,Pool Descr,Competency,Description,Capacité,Proficiency Description,Année obtention,Dernière année util.,Années expérience,Supervisor Name,Interest Level of Competencies
0,LPS00135569,Sharon Cooper,E,NaN,EUA41,SHA DCT DCUSTOMER ACA-BT 41,FR,BUGFRANCE,FRICE,L,...,FR G OUEST LR,201496,Microsoft PowerBI,0,3-Très bon,0,2020,1,Crystal Schultz,2
1,LPS00135569,Sharon Cooper,E,NaN,EUA41,SHA DCT DCUSTOMER ACA-BT 41,FR,BUGFRANCE,FRICE,L,...,FR G OUEST LR,202275,Sécurité opérationnelle,2,4-Expert,0,2023,0,Crystal Schultz,2
2,LPS00135569,Sharon Cooper,E,NaN,EUA41,SHA DCT DCUSTOMER ACA-BT 41,FR,BUGFRANCE,FRICE,L,...,FR G OUEST LR,201319,Chaîne de blocs (blockchain),3,1-Faible,0,0,2,Crystal Schultz,2
3,LPS00135569,Sharon Cooper,E,NaN,EUA41,SHA DCT DCUSTOMER ACA-BT 41,FR,BUGFRANCE,FRICE,L,...,FR G OUEST LR,201606,Google G Suite,0,4-Expert,0,0,1,Crystal Schultz,2
4,LPS00135569,Sharon Cooper,E,NaN,EUA41,SHA DCT DCUSTOMER ACA-BT 41,FR,BUGFRANCE,FRICE,L,...,FR G OUEST LR,201924,Sciences environnementales,0,2-Bon,0,2020,0,Crystal Schultz,2
5,LPS00136406,Matthew Wilson,N,NaN,EUA41,SHA D4B DATA FOR BUSINESS,FR,BUGFRANCE,FRICE,P,...,FR G SUD PACA,201054,Chaîne de blocs (blockchain),4,2-Bon,0,2020,1,Ray Johnson,2
6,LPS00136406,Matthew Wilson,N,NaN,EUA41,SHA D4B DATA FOR BUSINESS,FR,BUGFRANCE,FRICE,P,...,FR G SUD PACA,200373,Expérience utilisateur agile,4,1-Faible,0,2023,0,Ray Johnson,2
7,LPS00136406,Matthew Wilson,N,NaN,EUA41,SHA D4B DATA FOR BUSINESS,FR,BUGFRANCE,FRICE,P,...,FR G SUD PACA,202496,Gestion occasions d‘affaires,2,1-Faible,0,2020,5,Ray Johnson,2
8,LPS00136406,Matthew Wilson,N,NaN,EUA41,SHA D4B DATA FOR BUSINESS,FR,BUGFRANCE,FRICE,P,...,FR G SUD PACA,200473,Apache Solr,3,3-Très bon,0,2023,3,Ray Johnson,2
9,437182,Robert Trujillo,N,I,EUA41,SHA DCT DCUSTOMER SC-BT 50,FR,BUGFRANCE,FRICE,P,...,FR EUTM OPS,202107,Gitflow,2,2-Bon,0,0,2,Andrew Smith,3


In [3]:
# Joindre les deux dataframes avec la colonne 'Nom' comme clé
combined_df = pd.merge(coaff_df, psarm_df, on='Nom')

# Afficher les 10 premières lignes du dataframe combiné
combined_df.columns

Index(['Localisation', 'COEFF_F212', 'PROFIL', 'Nom', 'Missions_en_cours',
       'Competences', 'Date_Demarrage', 'Date_de_fin', 'Tx_occup', 'Stream_BT',
       'Code', 'Stat. Pers.', 'Type', 'HR Business Unit', 'Dept Descr',
       'Executive BU', 'CGI Groupe d'affaire', 'CGI BU', 'Statut employé',
       'Emploi', 'Location (CGI Office)', 'Classe', 'Étape', 'Type pmt GL',
       'Perm./temp.', 'Catégorie empl.', 'Proj Role', 'Home Country',
       'Home City', 'Home State/Prov', 'CGI Tenure',
       'Professional Experience Date', 'Location Type', 'Lieu Travail',
       'Code FCP', 'Pool Descr', 'Competency', 'Description', 'Capacité',
       'Proficiency Description', 'Année obtention', 'Dernière année util.',
       'Années expérience', 'Supervisor Name',
       'Interest Level of Competencies'],
      dtype='object')

In [4]:
resultat_dict = {}

grouped_df = combined_df.groupby(['Nom', 'COEFF_F212', 'PROFIL', 'Localisation', 'Stream_BT', 'Code', 'Supervisor Name'])

for (nom, coeff, profil, localisation, stream_bt, code, manager), group in grouped_df:
    availabilities = group.groupby(['Missions_en_cours', 'Competences', 'Date_Demarrage', 'Date_de_fin', 'Tx_occup'])
    competencies = group.groupby(['Description', 'Proficiency Description'])

    cle_principale = f"Nom: {nom}, Code: {code}, Coefficient: {coeff}, Profil: {profil}, Localisation: {localisation}, Equipe: {stream_bt}, Manager: {manager}"
        
    if cle_principale not in resultat_dict:
        resultat_dict[cle_principale] = {"Missions": [], "Compétences": []} 

    for (mission, competences, demarrage, fin, tx), availabilities in availabilities:
        if mission == 'DISPO ICE':
            key_value = f"Disponible à {int((float(tx))*100)}% du {demarrage} au {fin}"
        elif mission == "congés":
            key_value = f"En congés à {int((float(tx))*100)}% du {demarrage} au {fin}"
        else:
            key_value = f"Mission {competences} à {int((float(tx))*100)}% d'occupation chez {mission} du {demarrage} au {fin}"
    
        resultat_dict[cle_principale]["Missions"].append(key_value)

    for (description, proficiency), competencies in competencies:
        if proficiency == '1-Faible':
            key_value = f"Compétent en {description} à un niveau faible"
        elif proficiency == '2-Bon':
            key_value = f"Compétent en {description} à un niveau bon"
        elif proficiency == '3-Très bon':
            key_value = f"Compétent en {description} à un niveau très bon"
        elif proficiency == '4-Expert':
            key_value = f"Compétent en {description} à un niveau expert"

        resultat_dict[cle_principale]["Compétences"].append(key_value)

pprint(resultat_dict, width=200)

{'Nom: Aaron Kelly, Code: 470862, Coefficient: 150, Profil: Expert, Localisation: Mtp, Equipe: TALEND, Manager: Kelly Smith': {'Compétences': ['Compétent en Base à un niveau expert',
                                                                                                                                               'Compétent en Cacti Montitoring à un niveau faible',
                                                                                                                                               'Compétent en MS Dynamics CRM App. Dev. à un niveau '
                                                                                                                                               'faible',
                                                                                                                                               'Compétent en MS Dynamics CRM App. Dev. à un niveau '
                                                                          

In [5]:
# Exporter le dictionnaire en csv
resultat_df = pd.DataFrame(resultat_dict).T

# Renommer la première colonne en 'Membre'
resultat_df = resultat_df.rename_axis('Membres').reset_index()

# Convertir chaque colonne en chaîne de caractères avant la concaténation
resultat_df['Combined'] = resultat_df['Membres'].astype(str) + resultat_df["Missions"].astype(str) + resultat_df["Compétences"].astype(str)

# Replace "['" by ", Missions: ['" in the 'Combined' column
resultat_df['Combined'] = resultat_df['Combined'].str.replace('["', ', Missions: ["')
resultat_df['Combined'] = resultat_df['Combined'].str.replace("][", "], Compétences: [")

resultat_df.to_csv(combined_result_path)
pprint(resultat_df['Combined'][0])

('Nom: Aaron Kelly, Code: 470862, Coefficient: 150, Profil: Expert, '
 'Localisation: Mtp, Equipe: TALEND, Manager: Kelly Smith, Missions: ["Mission '
 'SSIS à 50% d\'occupation chez Engie du 2024-08-26 au 2024-10-07", "Mission '
 'SSIS à 50% d\'occupation chez Fraikin du 2024-05-29 au 2024-11-10", "Mission '
 'SSIS à 50% d\'occupation chez LVMH du 2024-06-20 au 2024-08-06", "Mission '
 'SSIS à 50% d\'occupation chez La Poste du 2024-09-06 au 2025-06-20", '
 '"Mission SSIS à 50% d\'occupation chez Société Générale du 2025-05-11 au '
 '2025-10-13"], Compétences: [\'Compétent en Base à un niveau expert\', '
 "'Compétent en Cacti Montitoring à un niveau faible', 'Compétent en MS "
 "Dynamics CRM App. Dev. à un niveau faible', 'Compétent en MS Dynamics CRM "
 "App. Dev. à un niveau faible', 'Compétent en Soutien prestation de services "
 "à un niveau expert']")
